# Stanford Natural Language Inference¶

В этой модели используются Elmo Embeddings, вместе с шестью слоями двунаправленных LSTM (biLSTM).
Accuraсy на тестовой выборке составляет около 75%. Результат можно улучшить, изменяя архитектуру модели(добавление новых слоев и тд), а также подольше ее обучив(мы обучали ее 7 часов). 

# Импорт библитоек

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from keras.callbacks import Callback
import tensorflow_hub as hub
import tensorflow as tf
import re

from keras import backend as K
import keras.layers as layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dropout, Dense, concatenate, Embedding, Flatten, Activation, SpatialDropout1D
from keras.layers import Bidirectional, GRU, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.optimizers import Adam
from keras.models import Model
from keras.utils import np_utils
from keras.engine import Layer
from keras import initializers, regularizers, constraints
from keras.layers import *

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.models import load_model
from keras.layers import LSTM, CuDNNGRU, CuDNNLSTM, Add, Reshape
from keras.layers import MaxPooling1D, Conv1D, MaxPooling1D, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from nltk.tokenize import sent_tokenize, word_tokenize

import warnings
warnings.filterwarnings('ignore')
import os
os.environ['OMP_NUM_THREADS'] = '4'


import re
import math
# set seed
np.random.seed(123)

Using TensorFlow backend.


# Чтение данных

In [2]:
train = pd.read_csv('snli_1.0/snli_1.0_train.txt', sep ='\t')
test = pd.read_csv('snli_1.0/snli_1.0_test.txt', sep ='\t')
valid = pd.read_csv('snli_1.0/snli_1.0_dev.txt', sep ='\t')

In [3]:
print("Обучение на", train.shape[0], "примерах")
print("Валидация на", test.shape[0], "примерах")
print("Тест на ", valid.shape[0], "примерах")
train[:10]

Обучение на 550152 примерах
Валидация на 10000 примерах
Тест на  10000 примерах


,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,label2,label3,label4,label5
0,neutral,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,3416050480.jpg#4,3416050480.jpg#4r1n,neutral,NaN,NaN,NaN,NaN
1,contradiction,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",3416050480.jpg#4,3416050480.jpg#4r1c,contradiction,NaN,NaN,NaN,NaN
2,entailment,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,"( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",3416050480.jpg#4,3416050480.jpg#4r1e,entailment,NaN,NaN,NaN,NaN
3,neutral,( Children ( ( ( smiling and ) waving ) ( at c...,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...,Children smiling and waving at camera,They are smiling at their parents,2267923837.jpg#2,2267923837.jpg#2r1n,neutral,NaN,NaN,NaN,NaN
4,entailment,( Children ( ( ( smiling and ) waving ) ( at c...,( There ( ( are children ) present ) ),(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...,Children smiling and waving at camera,There are children present,2267923837.jpg#2,2267923837.jpg#2r1e,entailment,NaN,NaN,NaN,NaN
5,contradiction,( Children ( ( ( smiling and ) waving ) ( at c...,( ( The kids ) ( are frowning ) ),(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (DT The) (NNS kids)) (VP (VBP are...,Children smiling and waving at camera,The kids are frowning,2267923837.jpg#2,2267923837.jpg#2r1c,contradiction,NaN,NaN,NaN,NaN
6,contradiction,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,( ( The boy ) ( ( ( skates down ) ( the sidewa...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ skate...,A boy is jumping on skateboard in the middle o...,The boy skates down the sidewalk.,3691670743.jpg#0,3691670743.jpg#0r1c,contradiction,NaN,NaN,NaN,NaN
7,entailment,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,( ( The boy ) ( ( does ( a ( skateboarding tri...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ does)...,A boy is jumping on skateboard in the middle o...,The boy does a skateboarding trick.,3691670743.jpg#0,3691670743.jpg#0r1e,entailment,NaN,NaN,NaN,NaN
8,neutral,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,( ( The boy ) ( ( is ( wearing ( safety equipm...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ is) (...,A boy is jumping on skateboard in the middle o...,The boy is wearing safety equipment.,3691670743.jpg#0,3691670743.jpg#0r1n,neutral,NaN,NaN,NaN,NaN
9,neutral,( ( An ( older man ) ) ( ( ( sits ( with ( ( h...,( ( An ( older man ) ) ( ( ( drinks ( his juic...,(ROOT (S (NP (DT An) (JJR older) (NN man)) (VP...,(ROOT (S (NP (DT An) (JJR older) (NN man)) (VP...,An older man sits with his orange juice at a s...,An older man drinks his juice as he waits for ...,4804607632.jpg#0,4804607632.jpg#0r1n,neutral,NaN,NaN,NaN,NaN


In [4]:
test[test['gold_label']=='-'].count()

gold_label                176
sentence1_binary_parse    176
sentence2_binary_parse    176
sentence1_parse           176
sentence2_parse           176
sentence1                 176
sentence2                 176
captionID                 176
pairID                    176
label1                    176
label2                    176
label3                    176
label4                    176
label5                    175
dtype: int64

In [5]:
train.isnull().sum()  # количество NaN элементов в каждом столбце

gold_label                     0
sentence1_binary_parse         0
sentence2_binary_parse         6
sentence1_parse                0
sentence2_parse                0
sentence1                      0
sentence2                      6
captionID                      0
pairID                         0
label1                         0
label2                    510782
label3                    510757
label4                    510769
label5                    513238
dtype: int64

# Предобработка данных
Удалим строки где метка равна "-" а также NA предложения

In [6]:
train = train.dropna(subset = ['sentence2'])
train = train[train["gold_label"] != "-"]
test = test[test["gold_label"] != "-"]
valid = valid[valid["gold_label"] != "-"]

In [7]:
train.nunique()

gold_label                     3
sentence1_binary_parse    150735
sentence2_binary_parse    479206
sentence1_parse           150735
sentence2_parse           479206
sentence1                 150735
sentence2                 479340
captionID                 151194
pairID                    549361
label1                         3
label2                         3
label3                         3
label4                         3
label5                         3
dtype: int64

In [8]:
# Преобразуем данные в нужный нам вид

def get_rnn_data(df):
    x = {
        'sentence1': df["sentence1"],
        #
        'sentence2': df["sentence2"],
        }
    return x

le = LabelEncoder()

X_train = get_rnn_data(train)
Y_train = np_utils.to_categorical(le.fit_transform(train["gold_label"].values)).astype("int64")

X_valid = get_rnn_data(valid)
Y_valid = np_utils.to_categorical(le.fit_transform(valid["gold_label"].values)).astype("int64")

X_test = get_rnn_data(test)
Y_test = np_utils.to_categorical(le.fit_transform(test["gold_label"].values)).astype("int64")

# NLI модель

In [9]:
class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)
    
#     def get_config(self):
#         config = {'output_dim': self.output_dim}
    
class NonMasking(Layer):   
    def __init__(self, **kwargs):   
        self.supports_masking = True  
        super(NonMasking, self).__init__(**kwargs)   
  
    def build(self, input_shape):   
        input_shape = input_shape   
  
    def compute_mask(self, input, input_mask=None):   
        # do not pass the mask to the next layers   
        return None   
  
    def call(self, x, mask=None):   
        return x   
  
    def get_output_shape_for(self, input_shape):   
        return input_shape
    
#     def get_config(self):
#         config = {'output_dim': self.output_dim}
        
custom_ob={'ElmoEmbeddingLayer': ElmoEmbeddingLayer, 'NonMasking': NonMasking}

In [10]:
#### Elmo attempt
def get_model():
    model = Sequential()
    inp1 = Input(shape=(1,), dtype="string", name="sentence1")
    inp2 = Input(shape=(1,), dtype="string", name="sentence2")
    
    def emb_layer(inp, col):
        x = ElmoEmbeddingLayer()(inp)
        return x

    x = concatenate([
                    emb_layer(inp1,"sen_1"),
                    emb_layer(inp2,"sen_2"),
                     ])
    
    x = NonMasking()(x)
    x = Reshape((1, 1024*2), input_shape=(1024*2,))(x)
    x = Bidirectional(LSTM(128, return_sequences=True,recurrent_dropout=0.2))(x)
    x = Bidirectional(LSTM(128, return_sequences=True,recurrent_dropout=0.2))(x)
    x = Bidirectional(LSTM(128, return_sequences=True,recurrent_dropout=0.2))(x)
    x = Bidirectional(LSTM(128, return_sequences=True,recurrent_dropout=0.2))(x)
    x = Bidirectional(LSTM(128, return_sequences=True,recurrent_dropout=0.2))(x)
    x = Bidirectional(LSTM(128, return_sequences=True,recurrent_dropout=0.2))(x)

    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])

    outp = Dense(3, activation="softmax", name="final_output")(x)
    
    model = Model(inputs=[inp1,inp2], outputs=outp)
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=0.001),
                  metrics=['accuracy'],
                 )

    return model

model = get_model()

model.summary()



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
keep_dims is deprecated, use keepdims instead


...

Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sentence1 (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
sentence2 (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
elmo_embedding_layer_1 (ElmoEmb (None, 1024)         4           sentence1[0][0]                  
__________________________________________________________________________________________________
elmo_embedding_layer_2 (ElmoEmb (None, 1024)         4           sentence2[0][0]                  
__________________________________________________________________________________________________
concatenat

# Колбеки

In [11]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=1, 
                                            verbose=1, 
                                            factor=0.5,
                                            min_lr=0.00001)
file_path="checkpoint_SNLI_weights.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max', save_weights_only=True)
early = EarlyStopping(monitor="val_acc", mode="max", patience=1)

model_callbacks = [checkpoint, early, learning_rate_reduction]

Здесь можно загрузить сюда любые веса моделей

In [12]:
model.load_weights('SNLI_6_LSTM_weights.hdf5') #Загрузим обученные ранее веса модели

# Обучение модели

In [13]:
# Пропустим обучение, т.к. мы уже загрузили обученные ранее веса 

# %%time
# model.fit(X_train, Y_train,
#           batch_size=128,
#           epochs=10,
#           verbose=2,
#           validation_data=(X_valid, Y_valid),
#           callbacks = model_callbacks
#          )

In [14]:
# model.save_weights("SNLI_weights.hdf5")
# model.save("SNLI_model.h5")

# Предсказание

In [15]:
%%time
test_pred = model.predict(X_test, batch_size=128)

Wall time: 10min 25s


In [16]:
test_acc = (np.argmax(test_pred, axis=1) == np.argmax(Y_test, axis=1)).sum()/Y_test.shape[0] * 100

print("Accuracy on test set is: %"+str(test_acc))

Accuracy on test set is: %75.61074918566774


# Сохранение предсказаний в файл

In [18]:
preds = np.argmax(test_pred, axis=1)
test['predicted_label'] = pd.DataFrame(preds)
test[['pairID','predicted_label']].to_csv('results.csv',index=False)